In [4]:
#%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation, cm
from IPython.display import HTML

In [5]:
num = 11
x = np.linspace(-50, 50, num)
y = np.linspace(-50, 50, num)
x, y = np.meshgrid(x, y)

In [6]:
z = x ** 2 + y ** 2

In [7]:
idx1 = np.array([[1 ,2], [2, 3]])
print(tuple(zip(*idx1)))
z[tuple(zip(*idx1))]

((1, 2), (2, 3))


array([ 2500.,  1300.])

In [8]:
z

array([[ 5000.,  4100.,  3400.,  2900.,  2600.,  2500.,  2600.,  2900.,
         3400.,  4100.,  5000.],
       [ 4100.,  3200.,  2500.,  2000.,  1700.,  1600.,  1700.,  2000.,
         2500.,  3200.,  4100.],
       [ 3400.,  2500.,  1800.,  1300.,  1000.,   900.,  1000.,  1300.,
         1800.,  2500.,  3400.],
       [ 2900.,  2000.,  1300.,   800.,   500.,   400.,   500.,   800.,
         1300.,  2000.,  2900.],
       [ 2600.,  1700.,  1000.,   500.,   200.,   100.,   200.,   500.,
         1000.,  1700.,  2600.],
       [ 2500.,  1600.,   900.,   400.,   100.,     0.,   100.,   400.,
          900.,  1600.,  2500.],
       [ 2600.,  1700.,  1000.,   500.,   200.,   100.,   200.,   500.,
         1000.,  1700.,  2600.],
       [ 2900.,  2000.,  1300.,   800.,   500.,   400.,   500.,   800.,
         1300.,  2000.,  2900.],
       [ 3400.,  2500.,  1800.,  1300.,  1000.,   900.,  1000.,  1300.,
         1800.,  2500.,  3400.],
       [ 4100.,  3200.,  2500.,  2000.,  1700.,  1600.,

In [9]:
def starting_point(shape):
    return np.array([np.random.randint(i) for i in shape])

In [33]:
def neighbors(point, shape):
    a = np.arange(-1, 2)
    ns = (point + np.stack(np.meshgrid(*[a for i in range(len(point))]), axis=-1)).reshape(-1, len(point))
    gt = lambda n: np.all(np.less(n, shape)) and np.all(np.greater(n, -1)) and 
    return ns[[gt(n) for n in ns]]
    
neighbors(np.array([0, 0]), (5, 5))

array([[0, 0],
       [1, 0],
       [0, 1],
       [1, 1]])

In [11]:
def greedy(f, fitter):
    c = starting_point(f.shape)
    better = True
    while better:
        better = False
        for n in neighbors(c):
            vc = f[tuple(c)]
            vn = f[tuple(n)]
            if fitter(vc, vn) == vn and fitter(vc, vn) != vc:
                yield n
                c = n
                better = True
                break

In [17]:
for i in greedy(z, max):
    print i

[-1  4]
[-1  3]
[-1  2]
[-1  1]
[-1  0]


In [24]:
fig = plt.figure()

ax = Axes3D(fig)
surf = ax.plot_surface(x, y, z, cmap=cm.coolwarm, linewidth=0, antialiased=False)
    
def init():
    ax.view_init(elev=10., azim=0)
    return (surf,)

def animate(i):
    ax.view_init(elev=10., azim=i)
    return (surf,)

matplotlib.rcParams['animation.writer'] = 'avconv'
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=360, interval=20, blit=True)
HTML(anim.to_html5_video())